In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import keras
import os

Using TensorFlow backend.


In [2]:
with io.open('Swami.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


corpus length: 637135


In [3]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
words = [word for word in tokens if word.isalpha()]
print(len(words))


112550


In [17]:
words = words[:10000]

In [18]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

print("word_indices", type(word_indices), "length:",len(word_indices) )
print("indices_words", type(indices_word), "length", len(indices_word))


word_indices <class 'dict'> length: 2269
indices_words <class 'dict'> length 10000


In [19]:
maxlen = 20
step = 3
print("maxlen:",maxlen,"step:", step)
sentences = []
next_words = []
next_words= []
sentences1 = []
list_words = []

sentences2=[]
list_words = words

maxlen: 20 step: 3


In [20]:
for i in range(0,len(list_words)-maxlen, step):
    sentences2 = ' '.join(list_words[i: i + maxlen])
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
print('nb sequences(length of sentences):', len(sentences))
print("length of next_word",len(next_words))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        #print(i,t,word)
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1


nb sequences(length of sentences): 3327
length of next_word 3327
Vectorization...


In [21]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(words))))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


In [63]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):

    file=open("output.txt", "a")
    start_index = random.randint(0, len(list_words) - maxlen - 1)
    for diversity in [ 1.0, 1.2]:
        file.write('\n')
        file.write('diversity is\n')
        file.write(str(diversity))
        file.write('\n')
        generated = ''
        sentence = list_words[start_index: start_index + maxlen]
        generated += ' '.join(sentence)
        file.write('Generated with seed\n')
        file.write(generated)
        file.write('\n')
        for i in range(500):
            x = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x[0, t, word_indices[word]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            generated += next_word
            del sentence[0] 
            sentence.append(next_word)
            file.write(' ')
            file.write(next_word)
        print()
    file.close()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(X, y, #Big X for words
          batch_size=256,
          epochs=300,
          callbacks=[print_callback])

Epoch 1/300
3072/3327 [==========================>...] - ETA: 0s - loss: 5.6696

3327/3327 [==============================] - 44s 13ms/step - loss: 5.6749
Epoch 2/300
3072/3327 [==========================>...] - ETA: 0s - loss: 5.6461

3327/3327 [==============================] - 44s 13ms/step - loss: 5.6533
Epoch 3/300
3072/3327 [==========================>...] - ETA: 0s - loss: 5.6179

3327/3327 [==============================] - 45s 14ms/step - loss: 5.6241
Epoch 4/300
3072/3327 [==========================>...] - ETA: 0s - loss: 5.5754

3327/3327 [==============================] - 45s 14ms/step - loss: 5.5926
Epoch 5/300
3072/3327 [==========================>...] - ETA: 0s - loss: 5.5920

3327/3327 [==============================] - 44s 13ms/step - loss: 5.5905
Epoch 6/300
3072/3327 [==========================>...] - ETA: 0s - loss: 5.5611

3327/3327 [==============================] - 44s 13ms/step - loss: 5.5653
Epoch 7/300
3072/3327 [==========================>...] - ETA: 0s - los